In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your video file

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
import os
import shutil
import cv2
from PIL import Image

# Load models
vehicle_model = YOLO("yolov8s.pt")
plate_model = YOLO("/content/drive/MyDrive/Vehicle-NumberPlateDetection /Number Plate/best.pt")

# Prepare output directory
save_folder = "/content/cropped_plates"
if os.path.exists(save_folder):
    shutil.rmtree(save_folder)
os.makedirs(save_folder, exist_ok=True)

# Setup video input and output
cap = cv2.VideoCapture("/content/test1.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/output_video.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

vehicle_classes = [2, 3, 5, 7]
coco_classes = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Optionally work with full-res (already used as-is)
    full_frame = frame.copy()
    vehicle_boxes = []

    # Detect and track vehicles
    result_vehicles = vehicle_model.track(full_frame, persist=True, conf=0.3)
    if result_vehicles[0].boxes.id is not None:
        for box in result_vehicles[0].boxes:
            class_id = int(box.cls[0])
            if class_id in vehicle_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                vehicle_id = int(box.id[0])
                label = coco_classes.get(class_id, "Vehicle")
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label} ID {vehicle_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                vehicle_boxes.append({
                    "id": vehicle_id,
                    "box": (x1, y1, x2, y2)
                })

    # Detect number plates
    result_plates = plate_model(full_frame)
    for box in result_plates[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        assigned_id = "Unknown"

        for v in vehicle_boxes:
            vx1, vy1, vx2, vy2 = v["box"]
            if vx1 <= cx <= vx2 and vy1 <= cy <= vy2:
                assigned_id = v["id"]
                break

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Plate of ID {assigned_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Crop, upscale & save as PNG
        plate_crop = full_frame[y1:y2, x1:x2]
        if assigned_id != "Unknown" and plate_crop.size > 0:
            plate_crop = cv2.resize(plate_crop, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
            filename = f"{save_folder}/plate_vid{assigned_id}_frame{frame_count:05d}.png"
            cv2.imwrite(filename, plate_crop)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()

print(f"✅ Done! Cleared folder and saved cropped plates to: {save_folder}")

In [ ]:
!pip install tensorflow

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image, ImageEnhance
import torch
import os
import shutil
import pandas as pd
import numpy as np
import cv2
import re
from collections import defaultdict, Counter
from difflib import SequenceMatcher
import tensorflow as tf

# Load TrOCR
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load Zero-DCE
zero_dce_model = tf.keras.models.load_model("/content/drive/MyDrive/Zero-DCE-data/zero_dce_model.keras", compile=False)

# Constants
plate_dir = "/content/cropped_plates"
preprocessed_dir = "/content/preprocessed_plates"
BRIGHTNESS_THRESHOLD = 100  # You can tune this

# Clear and recreate preprocessing folder
if os.path.exists(preprocessed_dir):
    shutil.rmtree(preprocessed_dir)
os.makedirs(preprocessed_dir, exist_ok=True)

image_files = sorted([
    f for f in os.listdir(plate_dir)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

pattern1 = re.compile(r"^[A-Z]{2}-\d{4}$")     # AB-1234
pattern2 = re.compile(r"^[A-Z]{3}\d{4}$")      # ABC1234

# Helper: check brightness
def compute_brightness(pil_img):
    gray = pil_img.convert('L')
    np_img = np.array(gray)
    return np.mean(np_img)

# Helper: apply zero-dce enhancement
def enhance_with_zero_dce(pil_img):
    img = pil_img.resize((224, 224))  # Resize for model
    img_np = np.array(img) / 255.0
    input_tensor = tf.convert_to_tensor(img_np, dtype=tf.float32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    enhanced_img = zero_dce_model(input_tensor)[0]
    enhanced_img = tf.clip_by_value(enhanced_img, 0, 1)
    enhanced_img = (enhanced_img[0].numpy() * 255).astype(np.uint8)
    return Image.fromarray(enhanced_img)

# Preprocess image (after optional DCE)
def preprocess_plate(image, save_path=None):
    width, height = image.size
    image = image.crop((int(width * 0.2), 0, width, height))  # Crop province

    # Upscale
    upscale_size = (int(image.width * 2), int(image.height * 2))
    image = image.resize(upscale_size, resample=Image.BICUBIC)

    # Convert to grayscale → histogram equalize → blur → sharpen
    gray = np.array(image.convert("L"))
    equalized = cv2.equalizeHist(gray)
    blurred = cv2.GaussianBlur(equalized, (3, 3), 0)

    sharp = ImageEnhance.Sharpness(Image.fromarray(blurred)).enhance(2.0)
    contrast = ImageEnhance.Contrast(sharp).enhance(1.5)

    processed_image = contrast.convert("RGB")

    if save_path is not None:
        processed_image.save(save_path)

    return processed_image

vehicle_reads = defaultdict(list)

for img_file in image_files:
    match = re.match(r"plate_vid(\d+)_frame\d+\.png", img_file)
    if not match:
        continue
    vehicle_id = match.group(1)
    img_path = os.path.join(plate_dir, img_file)

    image = Image.open(img_path).convert("RGB")

    brightness = compute_brightness(image)
    if brightness < BRIGHTNESS_THRESHOLD:
        image = enhance_with_zero_dce(image)

    save_path = os.path.join(preprocessed_dir, img_file)
    processed = preprocess_plate(image, save_path=save_path)

    pixel_values = processor(images=processed, return_tensors="pt").pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    cleaned = generated_text.replace(" ", "").upper()

    if pattern1.match(cleaned) or pattern2.match(cleaned):
        print(f"{img_file} → {cleaned} ✅ (valid)")
        vehicle_reads[vehicle_id].append((img_file, cleaned))
    else:
        print(f"{img_file} → {cleaned} ❌ (invalid)")

# Group and save
def group_similar(strings, threshold=0.75):
    groups = []
    for s in strings:
        placed = False
        for g in groups:
            if SequenceMatcher(None, s, g[0]).ratio() >= threshold:
                g.append(s)
                placed = True
                break
        if not placed:
            groups.append([s])
    return groups

final_rows = []

for vehicle_id, entries in vehicle_reads.items():
    if not entries:
        continue

    all_valid_texts = [text for _, text in entries]
    groups = group_similar(all_valid_texts, threshold=0.75)

    best_group = max(groups, key=lambda g: len(g))
    best_plate = Counter(best_group).most_common(1)[0][0]

    last_frame = sorted([img for img, text in entries if text in best_group])[-1]

    print(f"Vehicle {vehicle_id} → selected plate: {best_plate} (from {last_frame})")
    final_rows.append({"image": last_frame, "predicted_plate": best_plate})

df = pd.DataFrame(final_rows)
df.to_csv("/content/recognized_plates_grouped.csv", index=False)
print("✅ Final grouped results saved to recognized_plates_grouped.csv")

# ---------------------------------------------------------------
# 📊 Evaluation Metrics (Without Ground Truth)
# ---------------------------------------------------------------

# 1. Format Validity Rate
valid_total = sum(1 for entries in vehicle_reads.values() for _, text in entries if pattern1.match(text) or pattern2.match(text))
total_attempts = sum(len(entries) for entries in vehicle_reads.values())
valid_rate = valid_total / total_attempts if total_attempts > 0 else 0
print(f"\n📊 Format Validity Rate: {valid_rate:.2%} ({valid_total}/{total_attempts})")

# 2. Average Consistency per Vehicle
consistent_total = 0
for vehicle_id, entries in vehicle_reads.items():
    texts = [text for _, text in entries]
    if texts:
        most_common = Counter(texts).most_common(1)[0][1]
        consistency = most_common / len(texts)
        consistent_total += consistency
avg_consistency = consistent_total / len(vehicle_reads) if vehicle_reads else 0
print(f"📊 Average Consistency per Vehicle: {avg_consistency:.2%}")

# 3. Heuristic Quality Checks
invalid_chars = 0
short_or_long = 0
for vehicle_id, entries in vehicle_reads.items():
    for _, text in entries:
        if re.search(r"[^A-Z0-9\-]", text):  # unexpected characters
            invalid_chars += 1
        if len(text) < 6 or len(text) > 8:
            short_or_long += 1

print(f"📊 Heuristic Alert: {invalid_chars} outputs contain invalid characters.")
print(f"📊 Heuristic Alert: {short_or_long} outputs have unusual lengths.")


In [ ]:
from ultralytics import YOLO
import cv2
import os
import shutil

# Load models
vehicle_model = YOLO("yolov8s.pt")
plate_model = YOLO("/content/drive/MyDrive/Vehicle-NumberPlateDetection /Number Plate/best.pt")

# Prepare output directory
save_folder = "/content/cropped_plates"
if os.path.exists(save_folder):
    shutil.rmtree(save_folder)
os.makedirs(save_folder, exist_ok=True)

# Setup video input and output
cap = cv2.VideoCapture("/content/test1.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/output_video.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

vehicle_classes = [2, 3, 5, 7]
coco_classes = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}

frame_count = 0

# Placeholder lists to collect predictions and ground truths for evaluation
predicted_plates = []  # Each element: {"frame": int, "id": int, "box": (x1,y1,x2,y2)}
ground_truth_plates = []  # You must fill this with your actual GT data

def iou(boxA, boxB):
    # Compute Intersection over Union of two boxes
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou_val = interArea / float(boxAArea + boxBArea - interArea)
    return iou_val

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    full_frame = frame.copy()
    vehicle_boxes = []

    result_vehicles = vehicle_model.track(full_frame, persist=True, conf=0.3)
    if result_vehicles[0].boxes.id is not None:
        for box in result_vehicles[0].boxes:
            class_id = int(box.cls[0])
            if class_id in vehicle_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                vehicle_id = int(box.id[0])
                label = coco_classes.get(class_id, "Vehicle")
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label} ID {vehicle_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                vehicle_boxes.append({
                    "id": vehicle_id,
                    "box": (x1, y1, x2, y2)
                })

    result_plates = plate_model(full_frame)
    for box in result_plates[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        assigned_id = "Unknown"

        for v in vehicle_boxes:
            vx1, vy1, vx2, vy2 = v["box"]
            if vx1 <= cx <= vx2 and vy1 <= cy <= vy2:
                assigned_id = v["id"]
                break

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Plate of ID {assigned_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Crop, upscale & save as PNG
        plate_crop = full_frame[y1:y2, x1:x2]
        if assigned_id != "Unknown" and plate_crop.size > 0:
            plate_crop = cv2.resize(plate_crop, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
            filename = f"{save_folder}/plate_vid{assigned_id}_frame{frame_count:05d}.png"
            cv2.imwrite(filename, plate_crop)

        # Save prediction for evaluation
        predicted_plates.append({
            "frame": frame_count,
            "id": assigned_id,
            "box": (x1, y1, x2, y2)
        })

    out.write(frame)
    frame_count += 1

cap.release()
out.release()

print(f"✅ Done! Cleared folder and saved cropped plates to: {save_folder}")

# ----------------
# Evaluation metrics calculation
# ----------------

# For simplicity, let's compute counts for TP, FP, FN for plate detection per frame and vehicle id.

# Parameters
IOU_THRESHOLD = 0.5

# Convert ground_truth_plates and predicted_plates into dicts keyed by (frame, id) for quick access
gt_dict = {}
for gt in ground_truth_plates:
    gt_dict.setdefault((gt["frame"], gt["id"]), []).append(gt["box"])

pred_dict = {}
for pred in predicted_plates:
    pred_dict.setdefault((pred["frame"], pred["id"]), []).append(pred["box"])

TP = 0
FP = 0
FN = 0

for key in set(list(gt_dict.keys()) + list(pred_dict.keys())):
    gt_boxes = gt_dict.get(key, [])
    pred_boxes = pred_dict.get(key, [])

    matched_gt = set()
    matched_pred = set()

    for i, pbox in enumerate(pred_boxes):
        matched = False
        for j, gtbox in enumerate(gt_boxes):
            if j in matched_gt:
                continue
            if iou(pbox, gtbox) >= IOU_THRESHOLD:
                TP += 1
                matched_gt.add(j)
                matched_pred.add(i)
                matched = True
                break
        if not matched:
            FP += 1
    # Any ground truths not matched count as FN
    FN += (len(gt_boxes) - len(matched_gt))

# Avoid division by zero
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

print(f"Evaluation metrics:\nAccuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1-score: {f1_score:.4f}")
